In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import scipy.integrate as integrate
import ipyvolume as ipv
import importlib

In [ ]:
importlib.reload(ipv)

In [ ]:
%matplotlib widget

fig = plt.figure()
ax = fig.add_subplot(111)

def noise(x):
    return np.sin(x)

def fbm(octaves, lacunarity, gain, x):
    # initial values
    amplitude = 0.5;
    frequency = 1.;

    #  loop of octaves
    y = np.zeros_like(x)
    for i in range(int(octaves)):
        y += amplitude * noise(frequency*x);
        frequency *= lacunarity;
        amplitude *= gain;
    return y

def plot_fbm(octaves, lacunarity, gain):
    x = np.arange(-10, 10, 0.1)
    y = fbm(octaves, lacunarity, gain, x)
    ax.clear()
    ax.plot(x, y)

widgets.interact(plot_fbm, octaves=(1, 10, 1.0), lacunarity=(0, 10.0, 0.1), gain=(0, 2, 0.1))

In [ ]:
np.pi = 3.14
print(np.pi)

In [ ]:
2 ** 5

In [ ]:
import os
import sys
sys.path.insert(0, os.path.abspath('pycode'))

import spherical as sph
import BRDF as BRDF

import importlib
importlib.reload(sph)
importlib.reload(BRDF)

In [ ]:
alpha = np.arange(BRDF.MIN_GGX_ALPHA, 1, 0.01)
err = np.array([BRDF.NDF_NormalizationTest(BRDF.NDF_GGX, a, num_samples=512) for a in alpha])

In [ ]:
plt.plot(alpha, np.abs(err))
plt.ylim(0, 1)

In [ ]:
import os
import sys
sys.path.insert(0, os.path.abspath('pycode'))

import spherical as sph
import BRDF as BRDF

import importlib
importlib.reload(sph)
importlib.reload(BRDF)


alpha = np.arange(BRDF.MIN_GGX_ALPHA, 1, 0.2)
theta = np.arange(0, np.pi * 0.5, 0.4)
alpha, theta = np.meshgrid(alpha, theta)

err = np.zeros_like(alpha)
for i in range(alpha.shape[0]):
    for j in range(alpha.shape[1]):
        alpha_ = alpha[i][j]
        theta_ = theta[i][j]
        Wo = sph.spherical_dir(theta_, 0)
        cosThetaO = Wo[2]
        err[i][j] = BRDF.maskingFuncProjectedAreaTest(BRDF.NDF_GGX, BRDF.smithMaskingFunction, alpha_, Wo, num_samples=2048)/cosThetaO
        print(alpha_, theta_, err[i][j])


In [ ]:
ipv.clear()
ipv.plot_mesh(alpha, theta, np.abs(err))
ipv.show()